In [67]:
import ee
import rasterio
import numpy as np
import pandas as pd
from pprint import pprint
from IPython.display import display
from IPython.display import Image
import matplotlib.pyplot as plt
from matplotlib import colors

In [58]:
ee.Initialize()

In [59]:
from channel_classifier.internal.ee_utils import add_labels, compute_error_image, get_landsat_images, get_landsat_composite

In [60]:
def ndvi_l5(image):
    return image.normalizedDifference(['B4', 'B3']).rename(['NDVI'])

def ndvi_l8(image):
    return image.normalizedDifference(['B5', 'B4']).rename(['NDVI'])

In [64]:
path, row = [231, 71] # Mamore
bbox1 = [-63.89, -16.44, -63.47, -16.05]
bbox2 = [-64.47, -16.71, -64.05, -16.41]
start_date = '1984-01-01'
end_date = '2018-01-01'

In [65]:
region = ee.Geometry.Rectangle(bbox1)
background = ee.Geometry.Rectangle(bbox2)

def set_mean(image):
    d = image.reduceRegion(ee.Reducer.mean(), region)
    return image.set(d)
  
def set_background(image):
    d = image.reduceRegion(ee.Reducer.mean(), background)
    return image.set(d)

In [ ]:
fig, ax = plt.subplots(1,1)

images = ee.ImageCollection([])
ndvis = ee.ImageCollection([])
datasets = ['LANDSAT/LT05/C01/T1_TOA'] #, 'LANDSAT/LE07/C01/T1_TOA', 'LANDSAT/LC08/C01/T1_TOA']
functions = [ndvi_l5] #, ndvi_l5, ndvi_l8]
for dataset, ndvi in zip(datasets, functions):
    x = get_landsat_images(path, row, start_date, end_date, dataset_path=dataset)
    x = x.filter(ee.Filter.lt('CLOUD_COVER', 50))
    images = ee.ImageCollection(x)
    ndvis = images.map(ndvi).map(set_mean)
    control = images.map(ndvi).map(set_background)
    dates = images.aggregate_array('system:time_start').getInfo()
    y1 = ndvis.aggregate_array('NDVI').getInfo()
    y2 = control.aggregate_array('NDVI').getInfo()
    if len(y1) > 0:
        dates = pd.to_datetime(dates, unit='ms')
        data1 = pd.DataFrame({'NDVI' : y1}, index=dates)
        data2 = pd.DataFrame({'NDVI' : y2}, index=dates)
        data1 = data1.groupby(pd.TimeGrouper('A')).mean()
        data2 = data2.groupby(pd.TimeGrouper('A')).mean()
        print(data1)
        ax.plot(data1.index, data1['NDVI'], data2.index, data2['NDVI'], marker='s', markersize=5)
plt.legend(['Mamore tributary', 'Background'])